# Linear Classifiers in Python

#### Logistic Regression
* Logistic Regression is a linear classifier
* sklearn's Logistic Regression can also output confidence scores rather than "hard" or definite predictions with `.predict_proba()`

```
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.predict(X_test)
lr.score(X_test, y_test)
```

#### Using Linear SVC
* In sklearn the basic SVM classifier is called `LinearSVC()` or Linear Support Vector Classifier
* Note that sklearn's Logistic Regression and SVM implementations handle multiple classes (if a dataset has more than 2 classes) automatically.

#### Using SVC
* The SVC class fits a nonlinear SVM by default

* **Underfitting:** model is too simple, training accuracy low
* **Overfitting:** model is too complex, testing accuracy low